In [ ]:
"""when running this notebook in colab, you will need to install the following packages every session.  Sessions end after 8 or 12 hours.  
If you are running the notebook on your local computer, you will only need to do it once"""

'when running this notebook in colab, you will need to install the following packages every session.  Sessions end after 8 or 12 hours.  \nIf you are running the notebook on your local computer, you will only need to do it once'

In [ ]:
!python -m pip install --upgrade pprint
!python -m pip install --upgrade pyLDAvis
!python -m pip install --upgrade nltk
!python -m pip install --upgrade spacy
!python -m pip install --upgrade gensim
!python -m pip install --upgrade matplotlib
import nltk
nltk.download('popular')
#nltk.download('all')
#!python -m nltk.downloader all
import spacy.cli
spacy.cli.download("en_core_web_lg")
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint
     |████████████████████████████████| 1.7MB 8.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.3MB 278kB/s 
     |████████████████████████████████| 9.9MB 45.6MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136870 sha256=4273fa1560eeb9978bdcceeb27d50dfc9f29105bb34a323a02719eea32689f1f
  Stored in directory: /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2
Successfully built pyLDAvis
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.4 which is incom

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
--2021-04-27 14:26:53--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  12.2MB/s    in 1.3s    

2021-04-27 14:26:55 (12.2 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]

Archive:  mallet-2.0.8.zip
   creating: mallet-2.0.8/
   creating: mallet-2.0.8/bin/
  inflating: mallet-2.0.8/bin/classifier2info  
  inflating: mallet-2.0.8/bin/csv2classify  
  inflating: mallet-2.0.8/bin/csv2vectors  
  inflating: mallet-2.0.8/bin/mallet  
  inflating: mallet-2.0.8/bin/mallet.bat  
  inflating: mallet-2.0.8/bin/mallethon  
  inflating: mallet-2.0.8/bin/prepend-li

In [ ]:
from google.colab import files
#running this cell will prompt you to select the data file you wish to upload into your workspace
uploaded = files.upload()

Saving ansiedad.csv to ansiedad.csv


In [ ]:
#From https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
import io
"""you'll need to enter the file path for the data file you uploaded previously"""
df = pd.read_csv(io.BytesIO(uploaded['patientinfo_cholesterol.csv']))
df.head()

,submission_url,title,submission_text,comment
0,https://patient.info/forums/discuss/high-chole...,cholesterol,"hi, i have always had high cholesterol. i was ...",thank you for your reply Alice.
1,https://patient.info/forums/discuss/high-chole...,cholesterol,"hi, i have always had high cholesterol. i was ...",thank you for your reply Alice.
2,https://patient.info/forums/discuss/high-chole...,cholesterol,"hi, i have always had high cholesterol. i was ...",i cant believe the gp gave you statins with a ...
3,https://patient.info/forums/discuss/high-chole...,cholesterol,"hi, i have always had high cholesterol. i was ...",thank you for your reply Alice.
4,https://patient.info/forums/discuss/high-chole...,cholesterol,"hi, i have always had high cholesterol. i was ...",i cant believe the gp gave you statins with a ...


In [ ]:
 #In order to process the text in your file, you'll need to take the column(s) in your data that contain the comments and convert them into a python list.  
 #do this by setting the 'comments' variable to df.your_column_name.tolist()
 #if there are multiple columns you can add them together like this:  df.column_1.tolist() + df.column_2.tolist() etc.
 #IMPORTANT:  if one or more of your text columns contains duplicates, you will need to use the unique() function to keep only the unique values.  
 #This is done as follows:  df.your_column_name.unique().tolist()

 comments = df.title.unique().tolist() + df.submission_text.tolist() + df.comment.tolist()

In [ ]:

comments = df.title.unique().tolist() + df.submission_text.tolist() + df.comment.tolist()
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', str(sent)) for sent in comments]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

#pprint(data[:1])
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

#print(data_words[:1])

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[100]]])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        try:
          doc = nlp(" ".join(sent)) 
          texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        except:
          pass
    return texts_out

In [ ]:
# Remove Stop Words

data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Form Trigrams
data_words_trigrams = make_trigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_lg')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[['cholesterol']]
[[(0, 1)]]


In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.043*"take" + 0.029*"get" + 0.025*"stop" + 0.024*"go" + 0.024*"pain" + '
  '0.021*"well" + 0.019*"muscle" + 0.017*"effect" + 0.017*"good" + '
  '0.017*"think"'),
 (1,
  '0.043*"smoking" + 0.037*"smoke" + 0.021*"company" + 0.018*"big" + '
  '0.015*"pharma" + 0.013*"reply" + 0.009*"hard" + 0.009*"pill" + '
  '0.009*"trial" + 0.009*"doctor"'),
 (2,
  '0.040*"cholesterol" + 0.023*"people" + 0.023*"drug" + 0.020*"high" + '
  '0.018*"take" + 0.017*"effect" + 0.015*"give" + 0.014*"low" + 0.013*"side" + '
  '0.013*"heart"'),
 (3,
  '0.067*"statin" + 0.030*"know" + 0.028*"doctor" + 0.023*"say" + '
  '0.021*"level" + 0.018*"also" + 0.018*"year" + 0.018*"see" + 0.017*"blood" + '
  '0.017*"gp"')]


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.209847 -0.049824       1        1  36.731182
2     -0.098064 -0.216099       2        1  28.104583
3     -0.045647  0.314791       3        1  27.732116
1      0.353558 -0.048868       4        1   7.432120, topic_info=             Term          Freq         Total Category  logprob  loglift
18         statin  48711.000000  48711.000000  Default  30.0000  30.0000
0     cholesterol  23686.000000  23686.000000  Default  29.0000  29.0000
81            get  21518.000000  21518.000000  Default  28.0000  28.0000
13           know  16797.000000  16797.000000  Default  27.0000  27.0000
19           stop  18666.000000  18666.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
1314        local    773.844853    989.582844   Topic4  -5.2763   2.3534
544          call    876.159187   1417.970329   Topic4  -5.1522   2.1179
267        doctor   1380.893037  17469.296310   Topic4  -4.6972   0.0616
60           tell   1094.891732   9766.199891   Topic4  -4.9293   0.4111
536          work    866.948160   5894.419850   Topic4  -5.1627   0.6826

[220 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
819       4  0.998144    able
26        3  0.999742  advice
704       3  0.999538  advise
624       3  0.999823     age
2         3  0.999860     ago
...     ...       ...     ...
1363      4  0.997393   world
25        2  0.318290    year
25        3  0.666313    year
25        4  0.015391    year
686       4  0.997838     yet

[289 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 2])

In [ ]:
pyLDAvis.save_html(vis, 'everyday.html')